In [40]:
import cugraph

csv = 'data/min2d.csv'
data = cudf.read_csv(csv, names=['src', 'dst', 'wt'], dtype=['int32', 'int32', 'float32'])

G = cugraph.Graph()
G.from_cudf_edgelist(data, source='src', destination='dst', edge_attr='wt')

dist = cugraph.shortest_path(G, 1)
for i in range(len(dist)):
	print('vertex ' + str(dist['vertex'].iloc[i]) + ' distance is ' + str(dist['distance'].iloc[i]))

vertex 5 distance is 1.0
vertex 2 distance is 1.0
vertex 3 distance is 2.0
vertex 4 distance is 1.0
vertex 1 distance is 0.0
